In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('data.csv', parse_dates=[1]).sort_values(by = "date")
df['date'] = df['date'].dt.to_period('M')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180814 entries, 135026 to 180813
Data columns (total 37 columns):
 #   Column                           Non-Null Count   Dtype    
---  ------                           --------------   -----    
 0   optid                            180814 non-null  int64    
 1   date                             180814 non-null  period[M]
 2   secid                            180814 non-null  int64    
 3   cp_flag                          180814 non-null  object   
 4   strike                           180814 non-null  float64  
 5   bid                              180814 non-null  float64  
 6   ask                              180814 non-null  float64  
 7   volume                           180814 non-null  float64  
 8   openint                          180814 non-null  float64  
 9   impvol                           180814 non-null  float64  
 10  delta                            180814 non-null  float64  
 11  gamma                            1

In [4]:
df.head()

,optid,date,secid,cp_flag,strike,bid,ask,volume,openint,impvol,...,vix,dhedged_return_mid,dhedged_return_spot,dhedged_return_spot_gamma,dhedged_return_mid_delev,dhedged_return_spot_delev,dhedged_return_spot_gamma_delev,IV_mness_deriv_1,IV_ttm_deriv_1,short_rate
135026,10043558,1996-02,108105,P,575.0,0.3750,0.4375,20.0,2019.0,0.233068,...,15.37,-0.395369,-0.000248,-0.171185,-0.009710,-6.087413e-06,-0.004204,-0.037859,1.418111e-05,0.05427
421,10368325,1996-02,108105,C,500.0,155.2500,156.2500,0.0,750.0,0.424562,...,15.37,0.007059,0.001697,2.714935,0.001731,4.160267e-04,0.665643,NaN,NaN,0.05427
91308,11506611,1996-02,108105,P,665.0,23.8750,24.8750,52.0,459.0,0.122164,...,15.37,0.099590,0.003746,0.447423,0.006534,2.457917e-04,0.029355,-0.026410,-7.934008e-06,0.05427
112280,10170877,1996-02,108105,P,525.0,0.0625,0.1250,500.0,7725.0,0.310703,...,15.37,-0.117722,-0.000017,-0.061710,-0.003241,-4.689261e-07,-0.001699,-0.037859,1.418111e-05,0.05427
67985,11516393,1996-02,108105,C,685.0,7.8750,8.3750,0.0,458.0,0.128511,...,15.37,0.257634,0.003230,0.470434,0.011572,1.451034e-04,0.021131,-0.010402,8.521277e-07,0.05427


In [5]:
def get_summary(df_col):
    print(f'\nSummary of {df_col.name}:')
    print(f'Mean: {np.round( df_col.mean(),2)}')
    print(f'Median: {np.round( df_col.median(),2)}')
    print(f'Standard deviation: {np.round( df_col.std(),2)}')
    print(f'Number of observations: {np.round( df_col.count(),2)}')    

In [32]:
def add_months(date ,period_to_add=1):
    return (datetime.strptime(date, '%Y-%m') + relativedelta(months=period_to_add)).strftime('%Y-%m')

In [7]:
features = ['mness', 'ttm', 'embed_lev', 'impvol', 'gamma', 'vega', 'theta', 'midprice']

In [8]:
print('\nSummary statistics for Call options\n')
df[df["cp_flag"] == 'C'][features].apply(lambda x: get_summary(x), axis=0);


Summary statistics for Call options


Summary of mness:
Mean: 0.49
Median: 0.31
Standard deviation: 1.3
Number of observations: 69153

Summary of ttm:
Mean: 208.77
Median: 91.0
Standard deviation: 224.23
Number of observations: 69153

Summary of embed_lev:
Mean: 22.16
Median: 14.24
Standard deviation: 21.84
Number of observations: 69153

Summary of impvol:
Mean: 0.21
Median: 0.18
Standard deviation: 0.15
Number of observations: 69153

Summary of gamma:
Mean: 0.0
Median: 0.0
Standard deviation: 0.0
Number of observations: 69153

Summary of vega:
Mean: 240.56
Median: 175.74
Standard deviation: 230.0
Number of observations: 69153

Summary of theta:
Mean: -56.11
Median: -43.82
Standard deviation: 51.21
Number of observations: 69153

Summary of midprice:
Mean: 91.37
Median: 37.3
Standard deviation: 163.55
Number of observations: 69153


In [9]:
print('\nSummary statistics for Put options\n')

df[df["cp_flag"] == 'P'][features].apply(lambda x: get_summary(x), axis=0);


Summary statistics for Put options


Summary of mness:
Mean: -0.9
Median: -1.03
Standard deviation: 1.43
Number of observations: 111661

Summary of ttm:
Mean: 202.6
Median: 91.0
Standard deviation: 222.15
Number of observations: 111661

Summary of embed_lev:
Mean: 14.66
Median: 11.79
Standard deviation: 10.61
Number of observations: 111661

Summary of impvol:
Mean: 0.3
Median: 0.26
Standard deviation: 0.18
Number of observations: 111661

Summary of gamma:
Mean: 0.0
Median: 0.0
Standard deviation: 0.0
Number of observations: 111661

Summary of vega:
Mean: 178.79
Median: 104.99
Standard deviation: 205.16
Number of observations: 111661

Summary of theta:
Mean: -49.17
Median: -36.14
Standard deviation: 49.51
Number of observations: 111661

Summary of midprice:
Mean: 71.58
Median: 14.2
Standard deviation: 154.07
Number of observations: 111661


In [10]:
features += ['strike', 'spot_close', 'divrate', 'vix', 'short_rate', 'delta', 'date']# + ['optid']

In [11]:
features

['mness',
 'ttm',
 'embed_lev',
 'impvol',
 'gamma',
 'vega',
 'theta',
 'midprice',
 'strike',
 'spot_close',
 'divrate',
 'vix',
 'short_rate',
 'delta',
 'date']

In [49]:
df[features].tail()

,mness,ttm,embed_lev,impvol,gamma,vega,theta,midprice,strike,spot_close,divrate,vix,short_rate,delta,date
85034,-3.490754,56,11.845966,0.832513,0.000001,0.451954,-1.249111,0.025,825.0,2575.21,0.018232,9.97,0.012243,-0.000115,2017-10
85041,-3.480114,56,13.185075,0.751127,0.000001,0.498459,-1.243073,0.025,925.0,2575.21,0.018232,9.97,0.012243,-0.000128,2017-10
89125,1.858739,56,86.452661,0.077655,0.000881,68.390690,-17.360330,0.900,2725.0,2575.21,0.018232,9.97,0.012243,0.030214,2017-10
84988,-2.046977,56,27.136470,0.254423,0.000169,42.947040,-36.378740,1.650,2100.0,2575.21,0.018232,9.97,0.012243,-0.017387,2017-10
180813,-1.838982,56,30.074332,0.215465,0.000309,66.472960,-47.733300,2.500,2205.0,2575.21,0.018232,9.97,0.012243,-0.029196,2017-10


In [13]:
clear_df = df[features]

In [44]:
current_date = '2007-02'
y_price = clear_df[clear_df['date'] > current_date]['midprice']
y_price

50265      80.900
141595     86.400
5521      176.300
141621      1.475
6209        3.750
           ...   
85034       0.025
85041       0.025
89125       0.900
84988       1.650
180813      2.500
Name: midprice, Length: 132500, dtype: float64

In [45]:
add_months(current_date)

'2007-03'

In [47]:
first_df = clear_df[clear_df['date'] < current_date]
first_df.tail(45)

y_train = clear_df[clear_df['date'] == current_date]['midprice']
y_train

6348        4.25
6351       13.05
28319       1.80
72410       0.10
118765      8.80
           ...  
164545     16.80
141236    109.20
141258     30.60
6477        1.05
6418        4.60
Name: midprice, Length: 411, dtype: float64

In [51]:
first_df

,mness,ttm,embed_lev,impvol,gamma,vega,theta,midprice,strike,spot_close,divrate,vix,short_rate,delta,date
135026,-1.851039,28,40.719462,0.233068,0.001448,10.479270,-16.012020,0.40625,575.0,647.98,NaN,15.37,0.054270,-0.025529,1996-02
421,-1.838855,63,4.078667,0.424562,0.000625,13.046590,-33.536680,155.75000,500.0,647.98,NaN,15.37,0.054270,0.971854,1996-02
91308,0.361222,126,15.241659,0.122164,0.008373,147.084800,-14.929410,24.37500,665.0,647.98,NaN,15.37,0.054270,-0.573344,1996-02
112280,-2.445653,28,36.321439,0.310703,0.000276,2.663379,-5.490642,0.09375,525.0,647.98,NaN,15.37,0.054270,-0.005255,1996-02
67985,0.735826,126,22.263237,0.128511,0.006867,126.896900,-28.270680,8.12500,685.0,647.98,NaN,15.37,0.054270,0.279158,1996-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140898,1.971447,700,16.587442,0.103961,0.000631,257.184500,-9.800429,5.85000,1900.0,1430.50,0.020623,10.40,0.053642,0.067834,2007-01
50148,-0.617211,245,13.744014,0.151504,0.001529,316.975000,-25.533650,19.90000,1325.0,1430.50,0.020623,10.40,0.053642,-0.191196,2007-01
72250,-0.602407,56,35.585709,0.121717,0.004474,167.925400,-56.150670,9.20000,1390.0,1430.50,0.020623,10.40,0.053642,-0.228863,2007-01
164287,0.801262,518,11.532409,0.117313,0.001793,609.836500,-38.818240,41.60000,1600.0,1430.50,0.020623,10.40,0.053642,0.335371,2007-01


## Neural networks

In [2]:
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer0 = nn.Linear(13, 4)
#         self.layer1 = nn.Linear(1024, 256)
#         self.layer2 = nn.Linear(256, 64)
        self.out = nn.Linear(4, 1)

        self.act = nn.Softplus()

    def forward(self, x):
        x = self.layer0(x)
        x = self.act(x)
#         x = self.layer1(x)
#         x = self.act(x)
#         x = self.layer2(x)
#         x = self.act(x)
        x = self.out(x)
        return x



In [3]:
from torch.utils.data import Dataset
import torch

In [4]:
import torch
import numpy as np
# https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):

        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """

        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):

        score = val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            
        elif score > self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(
                f'Validation loss decreased ({self.val_loss_min:.8f} --> {val_loss:.8f}).  Saving model ...\n', val_loss - self.val_loss_min)
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [5]:
import torch
import torch.nn as nn

class Config:

    def __init__(self):

        # related to data
        self.path_to_dataset =  "/home/sgarkot/code/options/SP500_Options_Monthly.h5"

        # related to model
        self.num_epochs = 10
        # self.path = path
        self.device = torch.device("cpu")
#         self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.criterion = nn.MSELoss()

    def path(self, fold):

        return  "/home/sgarkot/code/options/DNN/model/fold_" + str(fold)

class OptionsDataset(Dataset):

    # load the dataset
    def __init__(self, path):

        # load the csv file as a dataframe
        df = pd.read_hdf(path).sort_values(by='date')
        df = df.reset_index()
        features = ['mness', 'ttm', 'embed_lev', 'impvol', 'gamma', 'vega', 'theta', 'strike', 'delta', 'spot_close'
                   , 'divrate', 'vix', 'short_rate']
        
        self.df = df 
        # store the inputs and outputs
        self.X = df[features].to_numpy()
        self.y = pd.DataFrame({'response': df['midprice']}).to_numpy()

        self.X = torch.tensor(self.X)
        self.y = torch.tensor(self.y)

        # convert to needed further types
        self.X = self.X.long()
        self.y = self.y.to(torch.float64)

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    def get_splits(self, fold):
        train_pc = 0.85

        date = "2007-01-"
        if fold == 1:
            date = "2012-01-"
        elif fold == 2:
            date = "2017-01-"

        series = self.df.date.apply(lambda x: str(x)).str.startswith(date)
        frame = {f"is_January_{date[:5]}": series}
        df = pd.DataFrame(frame)
        january_df = df.loc[df[f'is_January_{date[:5]}']==True]

        last_date_ind = int(list(january_df.index)[-1])
        train = [i for i in range(0, int(last_date_ind * train_pc))]
        test = [i for i in range(int(last_date_ind * train_pc), last_date_ind + 1)]

        return (train, test)

In [6]:
import torch
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

from tqdm import tqdm
import numpy as np
import pandas as pd
from numpy import vstack
from numpy import argmax

from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error, median_absolute_error, r2_score
from scipy.stats import median_absolute_deviation

def run_train_epoch(cfg, model, train_dl, criterion, optimizer):

    avg_loss = 0

    for i, (inputs, targets) in tqdm(enumerate(train_dl)):
        inputs = inputs.to(torch.float)
        targets = targets.to(torch.float)

        inputs = inputs.to(cfg.device)
        targets = targets.to(cfg.device)

        # clear the gradients
        optimizer.zero_grad()

        # compute the model output
        yhat = model(inputs)

        # calculate loss
        loss = criterion(yhat, targets)

        avg_loss += loss

        # credit assignment
        loss.backward()

        # update model weights
        optimizer.step()

    avg_loss = avg_loss / len(train_dl.dataset)

    return  avg_loss

def run_validation_epoch(cfg, model, test_dl, criterion, return_results=False):

    results = list()

    predictions, actuals = list(), list()

    loss = 0

    for i, (inputs, targets) in tqdm(enumerate(test_dl)):

        inputs = inputs.to(torch.float)
        targets = targets.to(torch.float)
        targets = targets.to(cfg.device)
        inputs = inputs.to(cfg.device)

        yhat = model(inputs)

        loss += criterion(yhat, targets)

        # retrieve numpy array
        yhat = yhat.detach().cpu().numpy()

        yhat = np.asarray(yhat)
        actual = targets.detach().cpu().numpy()

        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(actual), 1))
        
        # storing
        predictions.append(yhat)
        actuals.append(actual)

    loss = loss / len(test_dl.dataset)
    return loss

def train_for_a_fold(cfg, fold, dataset):

    PATH_TO_SAVE = cfg.path(fold)

    model = NeuralNet()
    model.to(cfg.device)

    criterion = cfg.criterion

    optimizer = Adam(model.parameters(), lr=0.001)

    stopper = EarlyStopping(patience=50, verbose=True, path = PATH_TO_SAVE + '/fold' + str(fold) + '_regression.pth')

    scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

    indices_train, indices_test = dataset.get_splits(fold)
    
    train_dl = DataLoader(Subset(dataset, indices_train), batch_size=32)
    test_dl = DataLoader(Subset(dataset, indices_test), batch_size=1024)

    for epoch in range(cfg.num_epochs):

        print("EPOCH:", epoch)

        train_loss = run_train_epoch(cfg, model, train_dl, criterion, optimizer)
        test_loss = run_validation_epoch(cfg, model, test_dl, criterion)

        stopper(train_loss, model)

        message = 'epoch {:d}/{:d}, train {} {:.4f}, \n Test {} {:.4f}, best test {} {:.4f}'.format(
                  epoch + 1, cfg.num_epochs, 'MSE', train_loss,
                  'MSE', test_loss,
                  'MSE', stopper.best_score)

        print(message)

        scheduler.step()

        if stopper.early_stop:
            break

    print("Model saved at " + stopper.path)

In [8]:
cfg = Config()
fold = 0
dataset = OptionsDataset(cfg.path_to_dataset)

In [ ]:
train_for_a_fold(cfg, fold, dataset)

In [ ]:
trains = [732.8655,704.4376,676.7267, 632.3918]